In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import vectorbt.generic.nb as nb
import talib 

import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
sys.path.append(function_essential_path)
import utils as us


ModuleNotFoundError: No module named 'utils'

In [ ]:
##FONCTION##

In [ ]:
##FONCTION LOGIQUE SIGNALS##

In [ ]:
##INDICATEUR FACTORY##

In [1]:
##CLASS STRATEGIE##

class Strategie():


    def __init__(self,data,tickers,frequence_exec,frequence,multitimeframe):
        self.data = data
        self.tickers = tickers
        self.frequence_exec = frequence_exec
        self.frequences = frequence
        self.multitimeframe = multitimeframe   ## Si 0 pas de multitimeframe si 1 multitimeframe

    def backtest(self):
        entries_long_df = pd.DataFrame()
        entries_short_df = pd.DataFrame()
        exits_long_df = pd.DataFrame()
        exits_short_df = pd.DataFrame()
        close_df = pd.DataFrame()

        for ticker in self.tickers:
            data = self.data[ticker]

            liste_value = self.make_value()
            long,short = self.make_entries()
            exits_long,exits_short = self.make_exits()

            #Reset les index pour tout normaliser
            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            exits_long = exits_long.reset_index(drop=True)
            exits_short = exits_short.reset_index(drop=True)
            close = data['close'].reset_index(drop=True)
            #enregistrer dans des dataframes 
            entries_long_df[ticker] = long
            entries_short_df[ticker] = short
            exits_long_df[ticker] = exits_long
            exits_short_df[ticker] = exits_short
            close_df[ticker] = close
        
        #remplacer les 0 qui ont état mis durant import data
        close_df = close_df.replace(0, None).fillna(method='ffill')
        portfolio = vbt.Portfolio.from_signals(
            close_df,
            entries_long_df,
            exits_long_df,
            entries_short_df,
            exits_short_df,
            freq=self.frequence_exec
        )
        us.generate_portfolio_report(portfolio,close_df)
        us.get_pnl(portfolio)
        return portfolio

    def optimize(self,metrics,choice,period1,period2,params1):
        dataframe_save = {}   # Dans le cadre d'un multi Asset ca va venir enregistrer le df de l'optimisation -> Puis faire un mean -> Voir le parametre qui fonctionne le mieux sur tous
        if choice == 1:
            entries_df = pd.DataFrame()
            exits_df = pd.DataFrame()
            for ticker in self.list_tickers:

                entries_df = entries_df[0:0]
                exits_df = exits_df[0:0]
                data = self.data[ticker]
                liste_value = self.make_value_no_combi()

                for i in range(params1.shape[1]):  ## Ici tu boucles sur le paramètre que tu veux opti / Il peut y avoir deux boucles si tu opti deux params !
                    params1_1D = params1.iloc[:, i]
                    liste_value[0] =  params1_1D
                    entries = self.make_entries()
                    exits = self.make_exits()

                    #Important a pas toucher
                    entries_df[i] = entries 
                    exits_df[i] = exits
                    #-----------------------#

                # Important a note !! Pour pas avoir d'erreur ensuite les noms des colonnes de entries et exits doivent avoir des index de 0 a n (0,1,2,3 etc)
                # Si tu as un soucis avec la suite apres le backtest regarde les indices de entries et exits

                close = data['close']
                close_df = close.replace(0, None).fillna(method='ffill')
                portfolio = vbt.Portfolio.from_signals(
                     close_df,
                     entries_df,
                     exits_df,
                     freq= self.frequence,
                     slippage=0.0005
                )
                combinaison = fe.get_combinaison(period1,period2)
                df = fe.get_df_optimisation_parameter(combinaison,portfolio,metrics)
                dataframe_save[ticker] = df



        if choice == 2:

            pass

    def make_entries(self):
        pass

    def make_exits(self):
        pass

    def make_value(self,data):
        if self.multitimeframe == 1:
            result = {}
            for tf in self.frequences:
                data = data[tf]
                # exemple :   rsi = vbt.RSI.run(close,windows=14)
                # result[f"{tf}+indicateur_name"] = rsi
            #une fois tout les indicateurs calculer tu dois les mettres sur le meme index, tu prends la valeurs la timeframe la plus petite et tu bases toute les autres sur celle ci
        else:
            pass

In [ ]:
##UTILISATION DE LA STRAT##